In [7]:
import dagshub
import mlflow
import pandas as pd

dagshub.init(repo_owner='TomC333', repo_name='ml-walmart-recruiting', mlflow=True)

test_X = pd.read_csv("data/test_lightGBM.csv")

model_name = "LightGBM_Walmart_LightGBM_V4"
model = mlflow.pyfunc.load_model(f"models:/{model_name}/latest")

predictions = model.predict(test_X)

submission = pd.read_csv("data/sampleSubmission.csv")
submission["Weekly_Sales"] = predictions

submission.to_csv("data/lightgbm_submission.csv", index=False)

print("Submission file created: data/lightgbm_submission.csv")


Initialized MLflow to track repo "TomC333/ml-walmart-recruiting"

Repository TomC333/ml-walmart-recruiting initialized!

[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
Submission file created: data/lightgbm_submission.csv
